# This notebook performs topic keyword extraction and topic coherence evaluation for the second hierarchical level of topic clustering.

## Steps:
1. **Preprocessing the text data**:
   - Loads the dataset `Concatenated_First_Second_Step_BERTopic_Result.csv`.
   - Preprocesses the text by removing stopwords and non-alphabetic tokens using NLTK, preparing the documents for keyword extraction.

2. **Keyword extraction with KeyBERT**:
   - Groups the documents by 'Higher_Topic_Label'.
   - Uses the KeyBERT model to extract the top 10 keywords for each topic (grouped by the second-level topic label).
   - The extracted keywords are stored in a dictionary and merged back into the original dataset.

3. **Coherence score calculation**:
   - Creates a Gensim dictionary from the processed documents.
   - For each topic, the c_v coherence score is calculated using Gensim's `CoherenceModel`.
   - The topic coherence scores, keywords, and topic labels are stored in a DataFrame and saved as `Second_Level_Topic_coherence.csv`.

4. **Results output**:
   - Displays the results, including second-level topic labels, keywords, and their coherence scores.
   - The average coherence score across all topics is calculated and displayed.


In [7]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keybert import KeyBERT
import nltk

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load your dataset
file_path = 'Concatenated_First_Second_Step_BERTopic_Result.csv'  # Replace with your file path
data = pd.read_csv(file_path)
data

[nltk_data] Downloading package punkt to /home/yc656703/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yc656703/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/w0/tmp/slurm_yc656703.50590673/ipykernel_80572/3315991922.py:13: DtypeWarning: Columns (0,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,...,Aspect1,Representative_Docs,Human_Readable_Topic,Higher_Topic_Label,Unnamed: 0_x,Second_Step_Topic_Name,Second_Step_Topic_Keywords,Second_Step_Topic_Representation,Second_Step_Representative_Docs,Unnamed: 0_y
0,806.1636,Ian Pratt-Hartmann,Ian Pratt-Hartmann,Data-Complexity of the Two-Variable Fragment w...,NaN,"Information and Computation, 207(8), 2009, pp....",10.1016/j.ic.2009.02.004,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['logic', 'satisfiability', 'logics', 'clause'...",[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN
1,808.0521,Ian Pratt-Hartmann,Ian Pratt-Hartmann and Lawrence S. Moss,Logics for the Relational Syllogistic,NaN,"Review of Symbolic Logic, 2(4), 2009, pp. 647-...",10.1017/S1755020309990086,NaN,cs.LO cs.CC cs.CL,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['logic', 'satisfiability', 'logics', 'clause'...",[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN
2,905.3108,Ian Pratt-Hartmann,Yevgeny Kazakov and Ian Pratt-Hartmann,A Note on the Complexity of the Satisfiability...,Full proofs for paper presented at the IEEE Co...,"Proceedings, 24th Annual IEEE Symposium on Log...",10.1109/LICS.2009.17,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['logic', 'satisfiability', 'logics', 'clause'...",[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN
3,1104.2444,Claus-Peter Wirth,Claus-Peter Wirth,A Simplified and Improved Free-Variable Framew...,ii + 114 pages,IfCoLog Journal of Logics and their Applicatio...,NaN,SEKI Report SR-2011-01,cs.AI math.LO,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['logic', 'satisfiability', 'logics', 'clause'...",[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN
4,1301.387,Pierfrancesco La Mura,Pierfrancesco La Mura,Game Networks,Appears in Proceedings of the Sixteenth Confer...,NaN,NaN,UAI-P-2000-PG-335-342,cs.GT cs.AI,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['games', 'player', 'equilibrium', 'game', 'eq...",[' We study zero-sum differential games with ...,Game Theory and Strategic Decision Making,Game Theory and Strategic Decision Making,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56547,2408.08302,Bin Hu,"Usman Syed, Ethan Light, Xingang Guo, Huan Zha...",Benchmarking the Capabilities of Large Languag...,NaN,NaN,NaN,NaN,cs.AI cs.CL cs.LG,http://arxiv.org/licenses/nonexclusive-distrib...,...,"['mobility', 'transportation', 'travel', 'traf...",[' Mobility service route design requires dem...,Transportation Systems and Mobility Analysis,Transportation and Mobility Analysis,20548.0,36_transportation_traffic_routes_ridesharing,"transportation, traffic, routes, ridesharing, ...","transportation (0.48), traffic (0.47), routes ...",Mobility service route design requires deman...,36.0
56548,2408.08307,Ahmed Imtiaz Humayun,"Ahmed Imtiaz Humayun, Ibtihel Amara, Candice S...",Understanding the Local Geometry of Generative...,"Pre-print. 11 pages main, 8 pages app., 28 fig...",NaN,NaN,NaN,cs.LG cs.CV,http://creativecommons.org/licenses/by-nc-sa/4.0/,...,"['latent', 'manifold', 'dimensional', 'spaces'...",[' Representing a manifold of very high-dimen...,Manifold Learning with Autoencoders,Geometric Deep Learning on Manifolds,20549.0,291_autoencoders_autoencoder_embeddings_encoder,"autoencoders, autoencoder, embeddings, encoder...","autoencoders (0.58), autoencoder (0.52), embed.

In [10]:
# Example: Assuming you have a separate list of documents
documents = data['text']

# Preprocess documents
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return ' '.join(tokens)  # Join tokens back to a single string for KeyBERT

processed_docs = [preprocess(doc) for doc in documents]

# Initialize KeyBERT model
kw_model = KeyBERT()

# Group documents by 'Second_Level_Topic_Label'
grouped_docs = data.groupby('Higher_Topic_Label')['text'].apply(lambda texts: ' '.join(texts))

# Extract keywords per topic
def extract_keywords(text, model, num_keywords=10):
    keywords = model.extract_keywords(text, stop_words='english', top_n=num_keywords)
    return [keyword[0] for keyword in keywords]

topic_keywords = {}
for topic_label, docs in grouped_docs.items():
    processed_text = preprocess(docs)
    keywords = extract_keywords(processed_text, kw_model)
    topic_keywords[topic_label] = keywords

# Convert topic_keywords dictionary to DataFrame
keywords_df = pd.DataFrame(list(topic_keywords.items()), columns=['Higher_Topic_Label', 'KeyBERT_Keywords'])

# Merge the KeyBERT keywords back into the original dataset
data_with_keywords = pd.merge(data, keywords_df, on='Higher_Topic_Label', how='left')

# Display the resulting dataset with the KeyBERT keywords added
data_with_keywords

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,...,Representative_Docs,Human_Readable_Topic,Higher_Topic_Label,Unnamed: 0_x,Second_Step_Topic_Name,Second_Step_Topic_Keywords,Second_Step_Topic_Representation,Second_Step_Representative_Docs,Unnamed: 0_y,KeyBERT_Keywords
0,806.1636,Ian Pratt-Hartmann,Ian Pratt-Hartmann,Data-Complexity of the Two-Variable Fragment w...,NaN,"Information and Computation, 207(8), 2009, pp....",10.1016/j.ic.2009.02.004,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN,"[satisfiability, formalizations, logics, forma..."
1,808.0521,Ian Pratt-Hartmann,Ian Pratt-Hartmann and Lawrence S. Moss,Logics for the Relational Syllogistic,NaN,"Review of Symbolic Logic, 2(4), 2009, pp. 647-...",10.1017/S1755020309990086,NaN,cs.LO cs.CC cs.CL,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN,"[satisfiability, formalizations, logics, forma..."
2,905.3108,Ian Pratt-Hartmann,Yevgeny Kazakov and Ian Pratt-Hartmann,A Note on the Complexity of the Satisfiability...,Full proofs for paper presented at the IEEE Co...,"Proceedings, 24th Annual IEEE Symposium on Log...",10.1109/LICS.2009.17,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN,"[satisfiability, formalizations, logics, forma..."
3,1104.2444,Claus-Peter Wirth,Claus-Peter Wirth,A Simplified and Improved Free-Variable Framew...,ii + 114 pages,IfCoLog Journal of Logics and their Applicatio...,NaN,SEKI Report SR-2011-01,cs.AI math.LO,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' We formulate discussion graph semantics of...,Formal Reasoning and Satisfiability in Logic,"Reasoning and Problem-Solving with Logic, Lang...",NaN,NaN,NaN,NaN,NaN,NaN,"[satisfiability, formalizations, logics, forma..."
4,1301.387,Pierfrancesco La Mura,Pierfrancesco La Mura,Game Networks,Appears in Proceedings of the Sixteenth Confer...,NaN,NaN,UAI-P-2000-PG-335-342,cs.GT cs.AI,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' We study zero-sum differential games with ...,Game Theory and Strategic Decision Making,Game Theory and Strategic Decision Making,NaN,NaN,NaN,NaN,NaN,NaN,"[strategyproof, strategic, strategies, games, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56547,2408.08302,Bin Hu,"Usman Syed, Ethan Light, Xingang Guo, Huan Zha...",Benchmarking the Capabilities of Large Languag...,NaN,NaN,NaN,NaN,cs.AI cs.CL cs.LG,http://arxiv.org/licenses/nonexclusive-distrib...,...,[' Mobility service route design requires dem...,Transportation Systems and Mobility Analysis,Transportation and Mobility Analysis,20548.0,36_transportation_traffic_routes_ridesharing,"transportation, traffic, routes, ridesharing, ...","transportation (0.48), traffic (0.47), routes ...",Mobility service route design requires deman...,36.0,"[migratory, migration, trajectories, traveling..."
56548,2408.08307,Ahmed Imtiaz Humayun,"Ahmed Imtiaz Humayun, Ibtihel Amara, Candice S...",Understanding the Local Geometry of Generative...,"Pre-print. 11 pages main, 8 pages app., 28 fig...",NaN,NaN,NaN,cs.LG cs.CV,http://creativecommons.org/licenses/by-nc-sa/4.0/,...,[' Representing a manifold of very high-dimen...,Manifold Learning with Autoencoders,Geometric Deep Learning on Manifolds,20549.0,291_autoencoders_autoencoder_embeddings_encoder,"autoencoders, autoencoder, embeddings, encoder...","autoencoders (0.58), autoencoder (0.52), embed...",Representing a manifold of very high-di

In [11]:
# Example: Assuming you have a separate list of documents
data = data_with_keywords.copy()
documents = data['text']

# Preprocess documents
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

processed_docs = [preprocess(doc) for doc in documents]

# Extract topics after filtering
# No need to split the 'KeyBERT_Keywords' column as it's already a list
topic_representations = data.groupby('Higher_Topic_Label')['KeyBERT_Keywords'].apply(lambda x: x.iloc[0])
topics = topic_representations.tolist()

# Calculate Topic Diversity
def calculate_topic_diversity(topics):
    unique_words = set()
    total_words = 0

    for topic in topics:
        unique_words.update(topic)  # Add words to the unique set
        total_words += len(topic)   # Count total words in all topics

    # Topic diversity is the proportion of unique words to total words
    topic_diversity = len(unique_words) / total_words if total_words > 0 else 0
    return topic_diversity

# Calculate topic diversity
topic_diversity = calculate_topic_diversity(topics)

# Display the topic diversity score
print(f"\nTopic Diversity: {topic_diversity}")



Topic Diversity: 0.5703846153846154


Topic Coherence

In [7]:
# Function to remove brackets and quotes, keeping only the words
def clean_column(text):
    # Remove square brackets and split by commas
    return text.strip("[]").replace("'", "")

# Apply the function to the DataFrame column
data['Representation'] = data['Representation'].apply(clean_column)
data

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,...,First_Step_Topic_Keywords,First_Step_Topic_Representation,First_Step_Representative_Docs,Aspect1,Count,Higher_Topic_Label,Human_Readable_Topic,Name,Representation,Representative_Docs
0,806.1636,Ian Pratt-Hartmann,Ian Pratt-Hartmann,Data-Complexity of the Two-Variable Fragment w...,NaN,"Information and Computation, 207(8), 2009, pp....",10.1016/j.ic.2009.02.004,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,"satisfiability, semantics, unsatisfiable, logi...","satisfiability (0.53), semantics (0.43), unsat...",We formulate discussion graph semantics of f...,"['logic', 'satisfiability', 'logics', 'clause'...",138,"Reasoning and Problem-Solving with Logic, Lang...",Formal Reasoning and Satisfiability in Logic,57_satisfiability_semantics_unsatisfiable_logics,"satisfiability, semantics, unsatisfiable, logi...",[' We formulate discussion graph semantics of...
1,808.0521,Ian Pratt-Hartmann,Ian Pratt-Hartmann and Lawrence S. Moss,Logics for the Relational Syllogistic,NaN,"Review of Symbolic Logic, 2(4), 2009, pp. 647-...",10.1017/S1755020309990086,NaN,cs.LO cs.CC cs.CL,http://arxiv.org/licenses/nonexclusive-distrib...,...,"satisfiability, semantics, unsatisfiable, logi...","satisfiability (0.53), semantics (0.43), unsat...",We formulate discussion graph semantics of f...,"['logic', 'satisfiability', 'logics', 'clause'...",138,"Reasoning and Problem-Solving with Logic, Lang...",Formal Reasoning and Satisfiability in Logic,57_satisfiability_semantics_unsatisfiable_logics,"satisfiability, semantics, unsatisfiable, logi...",[' We formulate discussion graph semantics of...
2,905.3108,Ian Pratt-Hartmann,Yevgeny Kazakov and Ian Pratt-Hartmann,A Note on the Complexity of the Satisfiability...,Full proofs for paper presented at the IEEE Co...,"Proceedings, 24th Annual IEEE Symposium on Log...",10.1109/LICS.2009.17,NaN,cs.LO cs.AI cs.CC,http://arxiv.org/licenses/nonexclusive-distrib...,...,"satisfiability, semantics, unsatisfiable, logi...","satisfiability (0.53), semantics (0.43), unsat...",We formulate discussion graph semantics of f...,"['logic', 'satisfiability', 'logics', 'clause'...",138,"Reasoning and Problem-Solving with Logic, Lang...",Formal Reasoning and Satisfiability in Logic,57_satisfiability_semantics_unsatisfiable_logics,"satisfiability, semantics, unsatisfiable, logi...",[' We formulate discussion graph semantics of...
3,1104.2444,Claus-Peter Wirth,Claus-Peter Wirth,A Simplified and Improved Free-Variable Framew...,ii + 114 pages,IfCoLog Journal of Logics and their Applicatio...,NaN,SEKI Report SR-2011-01,cs.AI math.LO,http://arxiv.org/licenses/nonexclusive-distrib...,...,"satisfiability, semantics, unsatisfiable, logi...","satisfiability (0.53), semantics (0.43), unsat...",We formulate discussion graph semantics of f...,"['logic', 'satisfiability', 'logics', 'clause'...",138,"Reasoning and Problem-Solving with Logic, Lang...",Formal Reasoning and Satisfiability in Logic,57_satisfiability_semantics_unsatisfiable_logics,"satisfiability, semantics, unsatisfiable, logi...",[' We formulate discussion graph semantics of...
4,1203.055,Afshin Rostamizadeh,"Corinna Cortes, Mehryar Mohri, Afshin Rostamiz...",Algorithms for Learning Kernels Based on Cente...,NaN,Journal of Machine Learning Research 13 (2012)...,NaN,NaN,cs.LG cs.AI,http://arxiv.org/licenses/nonexclusive-distrib...,...,"features, classification, learning, datasets, ...","features (0.37), classification (0.35), learni...",With the significant advancements of Large L...,"['data', 'learning', 'models', 'model', 'train...",20553,NaN,Multimodal Models for Image-Text Analysis,-1_features_classification_learning_datasets,"features, classification, learning, datasets, ...",[' With the significant advancements of Large...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55617,2408.08313,Weiyang Liu,"Zeju Qiu, Weiyang Liu, Haiwen Fe

In [12]:
import os
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import pandas as pd

# Disable Hugging Face tokenizers parallelism to avoid the warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Assuming 'processed_docs' is a list of tokenized documents from the previous code
# Create a Gensim dictionary from the processed documents
dictionary = Dictionary(processed_docs)

# Initialize a list to store the per-topic coherence c_v scores
per_topic_coherence_cv = []

# Calculate coherence for each topic in 'topics'
for topic in topics:
    # Create a list containing just the current topic
    current_topic = [topic]
    
    # Initialize the CoherenceModel for the current topic using 'c_v'
    coherence_model_cv = CoherenceModel(topics=current_topic, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    
    # Compute the c_v coherence score
    coherence_cv = coherence_model_cv.get_coherence()
    print(f"Current topic: {current_topic}")
    print(f"Coherence c_v: {coherence_cv}")
    
    # Append the c_v score to the list
    per_topic_coherence_cv.append(coherence_cv)
    
# Create a DataFrame to display the results
results = pd.DataFrame({
    "Second_Level_Topic_Label": topic_representations.index,  # Use topic labels from topic_representations
    "Keywords": topic_representations.values,  # Keywords from earlier processing
    "Coherence c_v": per_topic_coherence_cv,  # Coherence scores
})

# Display the results
print(results)

# Save the results to a CSV file
results.to_csv('Second_Level_Topic_coherence.csv', index=False)


Current topic: [['modeling', 'surfaces', 'curvatures', 'curvature', 'modelling', 'classifier', 'classifiers', 'supervised', 'surface', 'algorithms']]
Coherence c_v: 0.280821079731206
Current topic: [['articulated', 'posture', 'kinematic', 'kinect', 'pose', 'skeleton', 'poses', 'skeletal', 'routines', 'kinematics']]
Coherence c_v: 0.3286618923489777
Current topic: [['activelab', 'classifier', 'classifiers', 'datasets', 'classification', 'supervised', 'boosting', 'dataset', 'detects', 'classify']]
Coherence c_v: 0.44354651013325286
Current topic: [['machining', 'manufacturability', 'manufacturing', 'fabrication', 'milling', 'optimizes', 'industrial', 'steel', 'aluminum', 'machines']]
Coherence c_v: 0.3168083848555796
Current topic: [['chaotic', 'dynamic', 'machines', 'dynamical', 'reservoir', 'systems', 'algorithms', 'reservoirs', 'classifier', 'svm']]
Coherence c_v: 0.33555411307768
Current topic: [['modelling', 'modeling', 'modelled', 'models', 'forecasting', 'dynamic', 'simulations', 

In [13]:
file_path = 'Second_Level_Topic_coherence.csv'  # Replace with your file path
data = pd.read_csv(file_path)
mean_score = data['Coherence c_v'].mean()
data['Coherence c_v'].mean()
mean_score


0.3626645533580275